## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-26-13-03-01 +0000,nyt,Israel Faces Growing Pressure Over Hostages an...,https://www.nytimes.com/2025/08/26/world/middl...
1,2025-08-26-13-00-00 +0000,missionlocal,Sweet,https://missionlocal.org/2025/08/sf-photos-sweet/
2,2025-08-26-12-58-12 +0000,nyt,Rabbis Emerge as Growing Voice of Criticism of...,https://www.nytimes.com/2025/08/26/us/rabbis-g...
3,2025-08-26-12-53-00 +0000,nyt,2 Police Officers Killed in Shooting in Rural ...,https://www.nytimes.com/2025/08/26/world/austr...
4,2025-08-26-12-52-36 +0000,nypost,Video of Kilmar Abrego Garcia being detained b...,https://nypost.com/2025/08/26/us-news/kilmar-a...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2356/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
57,trump,61
164,new,14
133,cook,14
29,he,12
132,lisa,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
102,2025-08-26-03-44-00 +0000,wsj,"President Trump said he is removing Lisa Cook,...",https://www.wsj.com/politics/policy/trump-says...,156
77,2025-08-26-08-20-01 +0000,startribune,"Trump says he’s firing Fed Governor Lisa Cook,...",https://www.startribune.com/trump-fires-fed-go...,148
56,2025-08-26-09-30-00 +0000,wsj,President Trump’s attempt to fire Federal Rese...,https://www.wsj.com/economy/central-banking/un...,137
122,2025-08-26-01-20-12 +0000,nyt,Can Trump Fire a Fed Governor? What to Know as...,https://www.nytimes.com/2025/08/22/us/politics...,132
247,2025-08-25-17-11-00 +0000,wsj,President Trump intensified his tough-on-crime...,https://www.wsj.com/politics/policy/trump-exec...,129


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
102,156,2025-08-26-03-44-00 +0000,wsj,"President Trump said he is removing Lisa Cook,...",https://www.wsj.com/politics/policy/trump-says...
268,60,2025-08-25-15-40-13 +0000,nypost,Keurig Dr Pepper to buy JDE Peet’s in $18B dea...,https://nypost.com/2025/08/25/business/keurig-...
247,59,2025-08-25-17-11-00 +0000,wsj,President Trump intensified his tough-on-crime...,https://www.wsj.com/politics/policy/trump-exec...
13,46,2025-08-26-11-52-00 +0000,wsj,Illinois Democratic Gov. JB Pritzker promised ...,https://www.wsj.com/politics/policy/chicago-vo...
33,44,2025-08-26-10-39-53 +0000,cbc,Australia accuses Iran of being behind two ant...,https://www.cbc.ca/news/world/australia-iran-a...
225,41,2025-08-25-19-11-00 +0000,wsj,Kilmar Abrego Garcia was arrested by immigrati...,https://www.wsj.com/us-news/law/kilmar-abrego-...
187,40,2025-08-25-21-13-56 +0000,wapo,"Israeli attack on Gaza hospital kills 20, incl...",https://www.washingtonpost.com/world/2025/08/2...
216,37,2025-08-25-19-40-01 +0000,nypost,Trump claims ‘good conversation’ with Putin si...,https://nypost.com/2025/08/25/us-news/trump-cl...
273,32,2025-08-25-15-26-19 +0000,nypost,Trump signs executive order banning flag burni...,https://nypost.com/2025/08/25/us-news/trump-si...
105,30,2025-08-26-03-05-59 +0000,nypost,American tourist stabbed while defending women...,https://nypost.com/2025/08/25/world-news/ameri...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
